In [1]:
using RDatasets,Distributions,DecisionTree,MLBase
iris=dataset("datasets","iris")
features =convert(Array,iris[:,1:4])
labels=convert(Array,iris[:,5])

150-element Array{String,1}:
 "setosa"   
 "setosa"   
 "setosa"   
 "setosa"   
 "setosa"   
 "setosa"   
 "setosa"   
 "setosa"   
 "setosa"   
 "setosa"   
 "setosa"   
 "setosa"   
 "setosa"   
 ⋮          
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"

In [5]:
train=rand(Bernoulli(0.75),nrow(iris))
#convert(UInt8, x)
test=Bool[]
for i in 1:nrow(iris)
 push!(test,!train[i])
end

In [6]:
model=build_tree(labels[train],features[train,:])

Decision Tree
Leaves: 9
Depth:  5

In [7]:
model=prune_tree(model,0.7)
print_tree(model)

Feature 3, Threshold 2.45
L-> setosa : 37/37
R-> Feature 4, Threshold 1.75
    L-> Feature 3, Threshold 5.05
        L-> Feature 4, Threshold 1.65
            L-> versicolor : 34/34
            R-> Feature 2, Threshold 2.75
                L-> virginica : 1/1
                R-> versicolor : 1/1
        R-> virginica : 3/4
    R-> Feature 3, Threshold 4.85
        L-> Feature 1, Threshold 5.95
            L-> versicolor : 1/1
            R-> virginica : 2/2
        R-> virginica : 32/32


In [8]:
predictions = apply_tree(model,features[test,:])

38-element Array{String,1}:
 "setosa"    
 "setosa"    
 "setosa"    
 "setosa"    
 "setosa"    
 "setosa"    
 "setosa"    
 "setosa"    
 "setosa"    
 "setosa"    
 "setosa"    
 "setosa"    
 "setosa"    
 ⋮           
 "virginica" 
 "virginica" 
 "virginica" 
 "virginica" 
 "virginica" 
 "versicolor"
 "virginica" 
 "virginica" 
 "virginica" 
 "virginica" 
 "virginica" 
 "virginica" 

Performance Check

In [10]:
species=["setosa","virginica","versicolor"]
lm=labelmap(species)
labels_test=labelencode(lm,labels[test])
predictions_test=labelencode(lm, predictions)

38-element Array{Int64,1}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 2
 2
 2
 2
 2
 3
 2
 2
 2
 2
 2
 2

In [15]:
println(cor(labels_test,predictions_test)) #correlation coefficient
R2(labels_test,predictions_test) #R_squared test

0.9817853324553131


0.9615384615384616

In [16]:
for i in 1:length(labels_test)
    println(labels_test[i]," - " ,predictions_test[i])
end

1 - 1
1 - 1
1 - 1
1 - 1
1 - 1
1 - 1
1 - 1
1 - 1
1 - 1
1 - 1
1 - 1
1 - 1
1 - 1
3 - 3
3 - 3
3 - 3
3 - 3
3 - 3
3 - 3
3 - 3
3 - 3
3 - 3
3 - 3
3 - 3
3 - 3
3 - 3
2 - 2
2 - 2
2 - 2
2 - 2
2 - 2
2 - 3
2 - 2
2 - 2
2 - 2
2 - 2
2 - 2
2 - 2


we can see a single error of prediction in this set 2-3

In [23]:
r=roc(labels_test,predictions_test)

ROCNums{Int64}
  p = 38
  n = 0
  tp = 37
  tn = 0
  fp = 0
  fn = 0


In [25]:
println(true_positive_rate(r))
println(true_negative_rate(r))
println(precision(r))
println(recall(r))

0.9736842105263158
NaN
1.0
0.9736842105263158


In [ ]:
#we can repeat this for a range of different purities to get more precise results
accuracy_cv =DataFrame()
accuracy_cv[:purity]=collect(0.1:0.025:1.0)
accuracy_cv[:accuracy]= map(x->mean(nfoldCV_tree(labels[train],features[train,:],x,10)),accuracy_cv[:purity])
